In [3]:
import pandas as pd
import numpy as np
import joblib
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

In [4]:
#STCE
sheet_name1 = ['Sheet 1', 'Sheet 3', 'Sheet 7']

names1 = ['Electricity_import_GWh', 'Electricity_export_GWh', 'Available_electricity_internaly_GWh']
#STCG
sheet_name2 = ['Sheet 2', 'Sheet 6', 'Sheet 8', 'Sheet 12', 'Sheet 18']

names2 = ['Indigenous_production_TJ', 'Imports_TJ', 'Exports_TJ', 'Stock_Changes_TJ', 'Inland_consumption_TJ']
#STCS
sheet_name3 = ['Sheet 1', 'Sheet 2', 'Sheet 3', 'Sheet 4', 'Sheet 11', 'Sheet 12', 'Sheet 13',
            'Sheet 36', 'Sheet 37', 'Sheet 38','Sheet 41', 'Sheet 42', 'Sheet 43','Sheet 51', 'Sheet 52']

names3 = ['Hard_coal_production_kT', 'Brown_coal_production_kT', 'Coke_production_kT', 'Peat_production_kT',
        'Hard_coal_import_kT', 'Brown_coal_import_kT', 'Coke_import_kT', 'Change_in_stock_hard_coal_kT',
        'Change_in_stock_brown_coal_kT', 'Change_in_stock_coke_kT','Inland_deliveries_hard_coal_kT', 
        'Inland_deliveries_brown_coal_kT', 'Inland_deliveries_coke_kT','Transformation_hard_coal_kT',
        'Transformation_brown_coal_kT']

#Weather
sheet_name4 = ['Sheet 1', 'Sheet 2']
names4 = ['Heating_degree_days', 'Cooling_degree_days']

sheets = [sheet_name1, sheet_name2, sheet_name3, sheet_name4]
names = names1 + names2 + names3 + names4
file_names = ['data/STCE.xlsx','data/STCG.xlsx','data/STCS.xlsx','data/Weather.xlsx']

EU_Countries = ['Belgium',
'Bulgaria', 'Czechia','Denmark','Germany','Estonia','Greece','Spain','France','Croatia','Italy','Cyprus','Latvia',
'Lithuania','Luxembourg','Hungary','Malta','Netherlands','Austria','Poland','Portugal','Romania','Slovenia','Slovakia','Finland','Sweden']
# Ireland excluded because of missing data

def cleaning(data_frame):
    for_dropping = []
    for i in data_frame.columns: #checking for "unnamed" columns
        if 'Unnamed' in i:
            for_dropping.append(i)
    starting_line = np.where(data_frame['TIME'] == 'Belgium')[0][0] # select where table starts
    data_frame = data_frame.drop(for_dropping, axis = 1).iloc[starting_line:].reset_index(drop = True)
    data_frame = data_frame.T
    data_frame.columns = data_frame.iloc[0]
    data_frame = data_frame.fillna(0).replace(':', 0).drop('TIME')
    data_frame = data_frame.loc['2008-01':'2023-12',:]
    return data_frame

eilute = 9
data = []
for name, sheet in zip(file_names, sheets):
    for i in sheet:
        data_frame = pd.read_excel(name, sheet_name=i, skiprows=eilute)
        data_frame = cleaning(data_frame)
        data.append(data_frame)

pagal_salis = {}
for country in EU_Countries:
    temp = pd.DataFrame()
    for name, table in zip(names, data):
        temp[name] = table[country]
    pagal_salis[country] = temp

electricity_production = pd.read_csv('data/electricity_production.csv')
electricity_production.columns = ['Country','Time','Balance','Product','Value','Unit']
electricity_production = electricity_production.iloc[8:] 
electricity_production['Time'] = electricity_production['Time'].apply(pd.to_datetime)
production_grouped = electricity_production.groupby('Country') 

source = ['Coal Peat and Manufactured Gases', 'Oil and Petroleum Products', 'Natural Gas',
            'Total Renewables (Hydro Geo Solar Wind Other)', 'Nuclear']


consumption_by_country = {}
production_by_country = {}
for c in EU_Countries:
    if c == 'Czechia':
        country = 'Czech Republic'
    elif c == 'Slovakia':
        country = 'Slovak Republic'
    else:
        country = c
    temp = production_grouped.get_group(country)
    temp = temp[temp['Balance'] == 'Net Electricity Production'].drop('Balance', axis = 1)
    temp.set_index('Time', inplace=True)
    temp.index = temp.index.to_series().dt.strftime('%Y-%m')
    keys = temp['Product'].unique() # not all countries have same structure, for example there is no info about nuclear power if it is not used
    concatenated = pd.DataFrame()
    for s in source:
        if s in keys:
            concatenated = pd.concat([concatenated, temp[temp['Product'] == s]['Value']], axis = 1)
        else:
            concatenated[s] = 0
    concatenated.columns = source
    concatenated = concatenated.sort_index().loc[:'2023-12',:]
    production_by_country[c] = concatenated.fillna(0).astype('float32')

    concatenated = pd.DataFrame()
    temp = production_grouped.get_group(country)
    temp = temp[temp['Balance'] == 'Final Consumption (Calculated)'].drop('Balance', axis = 1)
    temp.set_index('Time', inplace=True)
    temp.index = temp.index.to_series().dt.strftime('%Y-%m')
    keys = temp['Product'].unique()
    concatenated = pd.DataFrame()
    concatenated = pd.concat([concatenated, temp[temp['Product'] == 'Electricity']['Value']], axis = 1)
    concatenated.columns = ['Electricity']
    concatenated = concatenated.sort_index().loc[:'2023-12',:]
    consumption_by_country[c] = concatenated.fillna(0).astype('float32')

commodities = {'Natural Gas':'NG=F', 'Oil':'BZ=F'}

natural_gas = yf.download(commodities['Natural Gas'], start='2008-01-01', end='2024-01-01')
oil = yf.download(commodities['Oil'], start='2008-01-01', end='2024-01-01')

coal = pd.read_csv('data/Coal.csv').drop('Volume', axis = 1)[::-1]
coal['Date'] = coal['Date'].apply(pd.to_datetime)
coal.set_index('Date', inplace=True)

natural_gas_monthly = natural_gas.resample('1m').agg({'Open':'first', 'High':'max', 'Low':'min', 'Close':'last'})
oil_monthly = oil.resample('1m').agg({'Open':'first', 'High':'max', 'Low':'min', 'Close':'last'})
coal_monthly = coal.resample('1m').agg({'Open':'first', 'High':'max', 'Low':'min', 'Close':'last'})

def average(x):
    return x[x>0].mean()

coal_monthly['average'] = coal_monthly.apply(average, axis=1)
natural_gas_monthly['average'] = natural_gas_monthly.apply(average, axis=1)
oil_monthly['average'] = oil_monthly.apply(average, axis=1)

coal_monthly.index = coal_monthly.index.to_series().dt.strftime('%Y-%m')
natural_gas_monthly.index = natural_gas_monthly.index.to_series().dt.strftime('%Y-%m')
oil_monthly.index = oil_monthly.index.to_series().dt.strftime('%Y-%m')

pred_countries = []
electricity = pd.read_csv('data/electricity.csv')
grouped = electricity.groupby('Country')
Electricity_prices_monthly = {}
for i in EU_Countries:
    try:
        temp = grouped.get_group(i)
    except:
        continue
    else:
        temp['Date'] = temp['Date'].apply(pd.to_datetime)
        temp.set_index('Date', inplace=True)
        temp = temp.drop('ISO3 Code', axis = 1)
        temp = temp.resample('1m').agg({'Price (EUR/MWhe)': 'mean'})
        temp.columns = ['Price']
        temp.index = temp.index.to_series().dt.strftime('%Y-%m')
        Electricity_prices_monthly[i] = temp.loc[:'2023-12',:]
        pred_countries.append(i)



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
def extraxt_features(Country):
    target = Electricity_prices_monthly[Country]
    target = target.apply(np.sqrt)
    training_data = pagal_salis[Country]
    training_data = pd.concat([training_data, production_by_country[Country]], axis = 1)
    training_data = pd.concat([training_data, consumption_by_country[Country]], axis = 1)
    training_data['Oil_price'] = oil_monthly.loc['2008-01':'2023-12']['average']
    training_data['Coal_price'] = coal_monthly.loc['2008-01':'2023-12']['average']
    training_data['Natural_gas_price'] = natural_gas_monthly.loc['2008-01':'2023-12']['average']


    training_data = training_data.fillna(0)
    for i in training_data.columns:
        temp = training_data[i].isnull()
        training_data[i][temp] = 0
    training_data = training_data.replace([np.inf, -np.inf], 0)

    period = 1
    starting = np.where(training_data.index.values == target.index[0])[0][0]-1
    ending = len(training_data) - 1
    shifted = []
    for p in range(1,period):
        temp = training_data.shift(p)
        names = []
        for name in temp.columns:
            names.append(f'{name}_{p}')
        temp.columns = names
        shifted.append(temp)
    X = pd.concat([training_data] + shifted, axis = 1)
    X = X.iloc[starting:ending]
    X = X.fillna(0)

    return (X, target)

In [6]:
def build_training_data(c):
    all = []
    for i in c:
        all.append(extraxt_features(i))

    X = pd.DataFrame()
    target = pd.DataFrame()
    for data, targ in all:
        X = pd.concat([X, data])
        target = pd.concat([target, targ])

    target = target.reset_index(drop=True)

    X['target'] = target.values
    X = X.sort_index()
    target = X['target'].reset_index(drop=True)
    X = X.drop('target', axis = 1)
    return X, target

In [7]:
X, target = build_training_data([i])
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.25, shuffle=False)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
model = LinearRegression()
model.fit(X_train, y_train)

coefficients_interaction = model.coef_
feature_importance_interaction = pd.DataFrame({
    'Feature': X.columns.values,
    'Coefficient': coefficients_interaction
}).sort_values(by='Coefficient', ascending=False)

print(feature_importance_interaction)

                                          Feature   Coefficient
12                            Hard_coal_import_kT  2.454851e+00
15                   Change_in_stock_hard_coal_kT  1.983067e+00
4                                      Imports_TJ  1.471714e+00
14                                 Coke_import_kT  7.180107e-01
17                        Change_in_stock_coke_kT  5.781354e-01
32                                     Coal_price  5.689184e-01
27                                    Natural Gas  4.674988e-01
2             Available_electricity_internaly_GWh  2.944569e-01
0                          Electricity_import_GWh  2.548654e-01
29                                        Nuclear  2.535005e-01
31                                      Oil_price  1.499859e-01
33                              Natural_gas_price  1.209307e-01
6                                Stock_Changes_TJ  9.499778e-02
5                                      Exports_TJ  9.048126e-02
30                                    El

In [9]:
new_columns = ['Coal Peat and Manufactured Gases', 'Oil and Petroleum Products',
       'Natural Gas', 'Total Renewables (Hydro Geo Solar Wind Other)',
       'Nuclear', 'Oil_price', 'Coal_price',
       'Natural_gas_price']

new_X = X[new_columns].copy()

new_X['Electricity_exchange_abroad_TWh'] = X['Electricity_import_GWh'] - X['Electricity_export_GWh']
new_X['Gas_exchange_abroad_TJ'] = X['Imports_TJ'] - X['Exports_TJ']
new_X['Fossile_fuel_production'] = X[['Hard_coal_production_kT', 'Brown_coal_production_kT', 'Coke_production_kT','Peat_production_kT']].sum(axis=1)
new_X['Fossile_fuel_import'] = X[['Hard_coal_import_kT', 'Brown_coal_import_kT', 'Coke_import_kT']].sum(axis=1)
new_X['Total_electricity_production_GWh'] = X[['Coal Peat and Manufactured Gases', 'Oil and Petroleum Products','Natural Gas',
                                               'Total Renewables (Hydro Geo Solar Wind Other)','Nuclear']].sum(axis=1)
new_X['From_renewables_pct'] = X['Total Renewables (Hydro Geo Solar Wind Other)']/new_X['Total_electricity_production_GWh']
new_X['From_coal_pct'] = X['Coal Peat and Manufactured Gases']/new_X['Total_electricity_production_GWh']
new_X['From_oil_pct'] = X['Oil and Petroleum Products']/new_X['Total_electricity_production_GWh']
new_X['From_gas_pct'] = X['Natural Gas']/new_X['Total_electricity_production_GWh']
new_X = new_X.drop(['Total_electricity_production_GWh'], axis = 1)
new_X = new_X.fillna(0)

X_train, X_test, y_train, y_test = train_test_split(new_X, target, test_size=0.25, shuffle=False)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
model = LinearRegression()
model.fit(X_train, y_train)

coefficients_interaction = model.coef_
feature_importance_interaction = pd.DataFrame({
    'Feature': new_X.columns.values,
    'Coefficient': coefficients_interaction
}).sort_values(by='Coefficient', ascending=False)

print(feature_importance_interaction)

                                          Feature   Coefficient
15                                   From_gas_pct  1.591405e+00
1                      Oil and Petroleum Products  1.474280e+00
6                                      Coal_price  6.689446e-01
5                                       Oil_price  2.448955e-01
8                 Electricity_exchange_abroad_TWh  1.434096e-01
9                          Gas_exchange_abroad_TJ  1.043083e-01
7                               Natural_gas_price  2.771826e-02
0                Coal Peat and Manufactured Gases  0.000000e+00
13                                  From_coal_pct  0.000000e+00
12                            From_renewables_pct -8.881784e-16
3   Total Renewables (Hydro Geo Solar Wind Other) -1.998401e-15
4                                         Nuclear -7.745558e-02
11                            Fossile_fuel_import -1.912309e-01
10                        Fossile_fuel_production -4.214016e-01
2                                     Na

In [11]:
training_data = {}
for i in pred_countries:
    X, target = build_training_data([i])
    
    new_columns = ['Coal Peat and Manufactured Gases', 'Oil and Petroleum Products',
       'Natural Gas', 'Total Renewables (Hydro Geo Solar Wind Other)',
       'Nuclear', 'Oil_price', 'Coal_price',
       'Natural_gas_price']

    new_X = X[new_columns].copy()

    new_X['Electricity_exchange_abroad_TWh'] = X['Electricity_import_GWh'] - X['Electricity_export_GWh']
    new_X['Gas_exchange_abroad_TJ'] = X['Imports_TJ'] - X['Exports_TJ']
    new_X['Fossile_fuel_production'] = X[['Hard_coal_production_kT', 'Brown_coal_production_kT', 'Coke_production_kT','Peat_production_kT']].sum(axis=1)
    new_X['Fossile_fuel_import'] = X[['Hard_coal_import_kT', 'Brown_coal_import_kT', 'Coke_import_kT']].sum(axis=1)
    new_X['Total_electricity_production_GWh'] = X[['Coal Peat and Manufactured Gases', 'Oil and Petroleum Products','Natural Gas',
                                                'Total Renewables (Hydro Geo Solar Wind Other)','Nuclear']].sum(axis=1)
    new_X['From_renewables_pct'] = X['Total Renewables (Hydro Geo Solar Wind Other)']/new_X['Total_electricity_production_GWh']
    new_X['From_coal_pct'] = X['Coal Peat and Manufactured Gases']/new_X['Total_electricity_production_GWh']
    new_X['From_oil_pct'] = X['Oil and Petroleum Products']/new_X['Total_electricity_production_GWh']
    new_X['From_gas_pct'] = X['Natural Gas']/new_X['Total_electricity_production_GWh']
    new_X = new_X.drop(['Total_electricity_production_GWh'], axis = 1)
    new_X = new_X.fillna(0)

    training_data[i] = (new_X, target)


In [12]:
joblib.dump(training_data, 'training_data.joblib')

['training_data.joblib']